In [1]:
import pandas as pd

import tensorflow
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout, Dense


In [13]:
symptom_data = pd.read_csv("../datasets/combined.csv")
media_data = pd.read_csv("../datasets/media_count_ratio_all_2021.csv")

In [53]:
symptom = "symptom:shortness of breath"


a = symptom_data[symptom]
b = media_data[symptom]
c = symptom_data["daily_new_positives"]


features = ["symptom", "media"]
target = "case"


df = pd.concat([a, b, c], axis=1)
df.columns = features + [target]
df

,symptom,media,case
0,0.65,0.002182,15074
1,0.69,0.000604,11368
2,0.70,0.001136,11209
3,0.77,0.001465,12666
4,0.74,0.001320,16648
...,...,...,...
360,0.80,0.000000,40780
361,0.82,0.000000,67090
362,0.83,0.025641,74207
363,0.83,0.000000,76555


In [54]:
test_start = int(365 * 0.8)

df_train = df.loc[:test_start].copy()
df_test = df.loc[test_start:].copy()


# test_start = int(365 * 0.8)

# df_train = df.loc[:test_start].copy()
# df_test = df.loc[test_start:].copy()

In [55]:
df_train

,symptom,media,case
0,0.65,0.002182,15074
1,0.69,0.000604,11368
2,0.70,0.001136,11209
3,0.77,0.001465,12666
4,0.74,0.001320,16648
...,...,...,...
288,0.64,0.001023,4204
289,0.64,0.004011,3003
290,0.72,0.001233,2892
291,0.73,0.000183,4668


In [56]:
X_train = df_train[["symptom", "media"]]
X_test = df_test[["symptom", "media"]]

y_train = df_train[["case"]]
y_test = df_test[["case"]]

print(X_train)
print(y_train)

     symptom     media
0       0.65  0.002182
1       0.69  0.000604
2       0.70  0.001136
3       0.77  0.001465
4       0.74  0.001320
..       ...       ...
288     0.64  0.001023
289     0.64  0.004011
290     0.72  0.001233
291     0.73  0.000183
292     0.72  0.000723

[293 rows x 2 columns]
      case
0    15074
1    11368
2    11209
3    12666
4    16648
..     ...
288   4204
289   3003
290   2892
291   4668
292   4223

[293 rows x 1 columns]


In [57]:
print(df_train["case"])



model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(1000, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation="relu"))

model.compile(
    loss=tensorflow.keras.losses.MeanSquaredLogarithmicError(), optimizer="adam"
)

print("model compiled")

print(model.summary())

0      15074
1      11368
2      11209
3      12666
4      16648
       ...  
288     4204
289     3003
290     2892
291     4668
292     4223
Name: case, Length: 293, dtype: int64
model compiled
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_22 (LSTM)              (None, 2, 50)             10400     
                                                                 
 dropout_18 (Dropout)        (None, 2, 50)             0         
                                                                 
 lstm_23 (LSTM)              (None, 1000)              4204000   
                                                                 
 dropout_19 (Dropout)        (None, 1000)              0         
                                                                 
 dense_7 (Dense)             (None, 1)                 1001      
                                                       

In [58]:
history = model.fit(X_train, y_train, epochs=1000, batch_size=16)

Epoch 1/1000
19/19 [==============================] - 4s 56ms/step - loss: 58.5172
Epoch 2/1000
19/19 [==============================] - 1s 54ms/step - loss: 32.8494
Epoch 3/1000
19/19 [==============================] - 1s 59ms/step - loss: 19.4690
Epoch 4/1000
19/19 [==============================] - 1s 57ms/step - loss: 15.7916
Epoch 5/1000
19/19 [==============================] - 1s 64ms/step - loss: 14.4526
Epoch 6/1000
19/19 [==============================] - 1s 60ms/step - loss: 13.6481
Epoch 7/1000
19/19 [==============================] - 1s 59ms/step - loss: 13.0117
Epoch 8/1000
19/19 [==============================] - 1s 58ms/step - loss: 12.4768
Epoch 9/1000
19/19 [==============================] - 1s 58ms/step - loss: 12.0144
Epoch 10/1000
19/19 [==============================] - 1s 58ms/step - loss: 11.6160
Epoch 11/1000
19/19 [==============================] - 1s 57ms/step - loss: 11.2562
Epoch 12/1000
19/19 [==============================] - 1s 60ms/step - loss: 10.9221
E

In [59]:
predictions = model.predict(X_test)

3/3 [==============================] - 2s 11ms/step


In [60]:
predictions

array([[5203.138  ],
       [5151.55   ],
       [2402.1028 ],
       [1920.7651 ],
       [2618.142  ],
       [5203.1416 ],
       [5151.5625 ],
       [5104.3765 ],
       [4396.0127 ],
       [1365.5122 ],
       [1362.9604 ],
       [ 677.22455],
       [4803.169  ],
       [3760.8564 ],
       [5151.5215 ],
       [5010.479  ],
       [1363.9559 ],
       [2150.0073 ],
       [4392.6426 ],
       [5221.5664 ],
       [5221.5615 ],
       [5104.259  ],
       [4802.924  ],
       [2617.87   ],
       [ 677.233  ],
       [2617.2622 ],
       [4394.2773 ],
       [5151.577  ],
       [5151.5    ],
       [4393.93   ],
       [2274.0166 ],
       [ 677.228  ],
       [3065.7654 ],
       [4400.87   ],
       [3069.997  ],
       [1920.3687 ],
       [ 628.21075],
       [1920.9846 ],
       [2149.879  ],
       [3760.9294 ],
       [5237.662  ],
       [5151.5254 ],
       [4802.6367 ],
       [5151.4824 ],
       [3065.7654 ],
       [2149.8462 ],
       [2618.171  ],
       [5010.

In [61]:
y_test

,case
292,4223
293,3594
294,4338
295,2861
296,3054
...,...
360,40780
361,67090
362,74207
363,76555
